In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

In [2]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = 15, 5
import numpy as np
np.random.seed(0)
from pyha import *
import scipy
from scipy import signal
import os
import inspect

import warnings
warnings.filterwarnings('ignore') # because they include system path -> fails unit tests on different computer
from pyha.simulation.simulation_interface import get_sorted_traces
simulations = ['MODEL', 'HARDWARE']

# Application

In [3]:
from pyha.cores import Spectrogram
from pyha.cores.util import *
plt.rcParams['figure.figsize'] = 15, 5

In [4]:
fft_size = 1024
avg_freq_axis = 8
avg_time_axis = 2

In [5]:
from pyha.cores import BitreversalFFTshiftAVGPool
file = os.path.dirname(inspect.getfile(BitreversalFFTshiftAVGPool)) + '/phantom3_low_power_bladerf.npy'
orig_inp = np.load(file) # IQ samples
orig_inp = orig_inp[:int(len(orig_inp)/fft_size)*fft_size] * 128

golden_output = Spectrogram(fft_size, avg_freq_axis, avg_time_axis).model(orig_inp)

# plt.title('Golden output')
# imshow(golden_output.T)

In [6]:
dut = Spectrogram(fft_size, avg_freq_axis, avg_time_axis)
sims0 = simulate(dut, orig_inp[:1024], simulations=simulations, trace=True)

INFO:sim:Enabled fast simulation (model cannot be converted to VHDL)
INFO:sim:Running "MODEL" simulation...


ZeroDivisionError: integer division or modulo by zero

In [ ]:
l = get_sorted_traces()
for k, v in l.items():
    plt.title(k)
    plt.plot(v[0])
    plt.plot(v[1])
    show_plot()
    plt.plot(v[0] - v[1])
    show_plot()
    print(snr(v[0], v[1]))
# plt.plot(l[0][0] - l[0][1])

# DC Removal

In [ ]:
# NBVAL_IGNORE_OUTPUT
from pyha.cores import DCRemoval
dut = DCRemoval(256, dtype=Complex)
sims0 = simulate(dut, orig_inp, simulations=simulations)

In [ ]:
snr(sims0['MODEL'], sims0['HARDWARE'])

In [ ]:
plt.title('Error')
plt.plot(sims0['MODEL'] - sims0['HARDWARE'])
show_plot()

# Packager

In [ ]:
from pyha.cores import Packager
dut = Packager(fft_size)
sims1 = simulate(dut, sims0['HARDWARE'], simulations=simulations)

In [ ]:
snr(sims1['MODEL'], sims1['HARDWARE'])

# Windower

In [ ]:
from pyha.cores import Windower
dut = Windower(fft_size, coefficient_bits=8)
sims2 = simulate(dut, sims1['HARDWARE'], simulations=simulations)

In [ ]:
snr(sims2['MODEL'], sims2['HARDWARE'])

In [ ]:
plt.title('Error')
plt.plot(sims2['MODEL'][0] - sims2['HARDWARE'][0])
show_plot()

# FFT

In [ ]:
from pyha.cores import R2SDF
dut = R2SDF(fft_size, twiddle_bits=9)
sims3 = simulate(dut, sims2['HARDWARE'], simulations=simulations)

In [ ]:
snr(sims3['MODEL'], sims3['HARDWARE'])

In [ ]:
plt.title('Error')
plt.plot(sims3['MODEL'].flatten() - sims3['HARDWARE'].flatten())
show_plot()

# FFT to power

In [ ]:
from pyha.cores import FFTPower
dut = FFTPower()
sims4 = simulate(dut, sims3['HARDWARE'], simulations=simulations)

In [ ]:
snr(sims4['MODEL'], sims4['HARDWARE'])

# Bitrverse, FFTShift, AVGPool

In [ ]:
from pyha.cores import BitreversalFFTshiftAVGPool
dut = BitreversalFFTshiftAVGPool(fft_size, avg_freq_axis, avg_time_axis)
sims5 = simulate(dut, sims4['HARDWARE'], simulations=simulations)

In [ ]:
snr(golden_output, sims5['HARDWARE'])

In [ ]:
imshow(golden_output.T)

In [ ]:
imshow(sims5['HARDWARE'].T)

In [ ]:
plt.plot(np.log10(golden_output[40])*10)
plt.plot(np.log10(sims5['HARDWARE'][40])*10)
show_plot()

In [ ]:
plt.title('Error')
plt.plot(golden_output.flatten() - sims5['HARDWARE'].flatten())
show_plot()